In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [3]:
# Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [5]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 53s 17ms/step - loss: 6.2535 - accuracy: 0.0759
Epoch 2/100
3010/3010 [==============================] - 53s 18ms/step - loss: 5.5164 - accuracy: 0.1233
Epoch 3/100
3010/3010 [==============================] - 53s 17ms/step - loss: 5.1367 - accuracy: 0.1464
Epoch 4/100
3010/3010 [==============================] - 54s 18ms/step - loss: 4.8090 - accuracy: 0.1640
Epoch 5/100
3010/3010 [==============================] - 54s 18ms/step - loss: 4.5059 - accuracy: 0.1809
Epoch 6/100
3010/3010 [==============================] - 52s 17ms/step - loss: 4.2258 - accuracy: 0.2021
Epoch 7/100
3010/3010 [==============================] - 52s 17ms/step - loss: 3.9588 - accuracy: 0.2240
Epoch 8/100
3010/3010 [==============================] - 53s 18ms/step - loss: 3.7062 - accuracy: 0.2532
Epoch 9/100
3010/3010 [==============================] - 54s 18ms/step - loss: 3.4661 - accuracy: 0.2842
Epoch 10/100
3010/3010 [==============================]

In [9]:
seed_text = "I will leave if they"
next_words = 3

In [10]:
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word


1/1 [==============================] - 0s 11ms/step


In [11]:
print(seed_text)

I will leave if they may help from
